In [8]:
import pandas as pd
import sqlite3
import time


In [13]:


# Define a function to determine the SeasonStage
def determine_season_stage(date, round_info):
    if "Apertura" in round_info and "Regular Season" in round_info:
        return "Apertura"
    elif "Clausura" in round_info and "Regular Season" in round_info:
        return "Clausura"
    elif "Guardianes" in round_info:
        return "Guardianes"
    else:
        return "Liguilla"

# Define a function to extract the year
def extract_year(round_info, date):
    if "Quarter-finals" in round_info or "Semi-finals" in round_info or "Repechaje" or "Reclasificacion" in round_info or "Finals" in round_info:
        year = pd.to_datetime(date).year
        year = str(year)
        return year
    return ''.join(filter(str.isdigit, round_info))

# Define a function to determine the Season Type
def determine_season_type(round_info):
    if len(round_info.split('— ')) > 1:
        return round_info.split('— ')[-1]
    if len(round_info.split(' ')) > 1:
        return round_info.split(' ')[-2] + " " + round_info.split(' ')[-1]
    return round_info.split(' ')[-1]

# Función para extraer la temporada del enlace
def extraer_temporada(link):
    temporada = link.split('/')[-2]
    return temporada

def fbref_pull_and_store_data(metadata):
    # Procesar cada fila y actualizar/crear bases de datos SQL
    # complete this ditionary with the metadata of the tables you want to update
    unique_teams  = metadata.MetaEquipo.unique().tolist()
    datos_equipo = []  # Inicializar el DataFrame para los datos del equipo
    data_dict = {x : metadata[metadata.MetaEquipo == x] for x in unique_teams}

    for _, grupo in data_dict.items():
        print(f"Procesando base de datos: ")

        for _, fila in grupo.iterrows():
            tabla_nombre = fila['Tabla']
            link = fila['Link']
            metaequipo = fila['MetaEquipo']
            temporada = extraer_temporada(link) 

            print(f"Descargando datos de {tabla_nombre}, {metaequipo}.{temporada} desde {link}")
            
            time.sleep(3)  # Espera 1 segundo entre cada solicitud
            datos_tabla = pd.read_html(link)
            datos_tabla = datos_tabla[1]
            datos_tabla['Temporada'] = temporada  # Agregar la temporada como columna
            datos_tabla['MetaEquipo'] = metaequipo
            
            datos_tabla.columns = datos_tabla.columns.str.replace(' ', '_') # Reemplazar espacios con guiones bajos 
            # datos_tabla.columns = datos_tabla.columns.droplevel(0)
            datos_equipo.append(datos_tabla)
    return pd.concat(datos_equipo, ignore_index=True, sort=False)


In [14]:
metadata = pd.read_csv("data/csvdata/metadata.csv")

retrieved_data = fbref_pull_and_store_data(metadata)

Procesando base de datos: 
Descargando datos de scores_and_fixtures, Barcelona-2023-2024 desde https://fbref.com/en/squads/206d90db/2023-2024/Barcelona-Stats#all_matchlogs
Descargando datos de scores_and_fixtures, Barcelona-2022-2023 desde https://fbref.com/en/squads/206d90db/2022-2023/Barcelona-Stats#all_matchlogs
Descargando datos de scores_and_fixtures, Barcelona-2021-2022 desde https://fbref.com/en/squads/206d90db/2021-2022/Barcelona-Stats#all_matchlogs
Descargando datos de scores_and_fixtures, Barcelona-2020-2021 desde https://fbref.com/en/squads/206d90db/2020-2021/Barcelona-Stats#all_matchlogs
Descargando datos de scores_and_fixtures, Barcelona-2019-2020 desde https://fbref.com/en/squads/206d90db/2019-2020/Barcelona-Stats#all_matchlogs
Descargando datos de scores_and_fixtures, Barcelona-2018-2019 desde https://fbref.com/en/squads/206d90db/2018-2019/Barcelona-Stats#all_matchlogs
Descargando datos de scores_and_fixtures, Barcelona-2017-2018 desde https://fbref.com/en/squads/206d90d

HTTPError: HTTP Error 429: Too Many Requests